# Slab, cylindrical, spherical and mixed confinement

In [1]:
def shape(confinement):
    if confinement == "slab":
        shape = """Slab dimension 2 bound0 -1 bound1 1"""
    elif confinement == "cylinder":
        shape = """Cylinder radius 4 first_point f f0 0 f1 0 f2 0 \
second_point s s0 0 s1 0 s2 1"""
    elif confinement == "sphere":
        shape = """Sphere radius 4"""
    elif confinement == "union":
        shape = """Sphere radius 2.5
union Slab dimension 2 bound0 -1 bound1 1"""
    elif confinement == "network":
        shape = """Sphere radius 5
union Cylinder radius 2 first_point f f0 0 f1 0 f2 0 second_point s s0 1 s1 0 s2 0
union Cylinder radius 2 first_point f f0 0 f1 0 f2 0 second_point s s0 0 s1 1 s2 0
union Cylinder radius 2 first_point f f0 0 f1 0 f2 0 second_point s s0 0 s1 0 s2 1"""
    else:
        assert False # unrecognized
    with open(confinement+'.txt', 'w') as file: file.write(shape)

for confinement in ["slab", "cylinder", "sphere", "union", "network"]:
    shape(confinement)
    params={"confinement": confinement, "cubic_box_length": 8, "num_particles": 50}
    if confinement == 'network':
        params["cubic_box_length"]=20
        params["num_particles"]=500
    script="""
MonteCarlo
Configuration cubic_box_length {cubic_box_length} particle_type0 /feasst/forcefield/lj.fstprt
Potential Model LennardJones
Potential VisitModel LongRangeCorrections
Potential Model ModelHardShape file_name {confinement}.txt
ThermoParams beta 1.5 chemical_potential0 1
Metropolis
TrialTranslate weight 1 tunable_param 2
TrialAdd particle_type 0
Run until_num_particles {num_particles}
RemoveTrial name TrialAdd
Log trials_per 1e4 file_name {confinement}.log
Movie trials_per 1e4 file_name {confinement}.xyz
CheckEnergy trials_per 1e4
Tune
Run num_trials 1e5
""".format(**params)

    with open('script.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("../../../build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
    with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)

# FEASST version: v0.19.0-95-g62bcb048ca-dirty-hwh/newtutorials
Configuration cubic_box_length 8 particle_type0 /home/hwh/feasst/forcefield/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
Potential Model ModelHardShape file_name slab.txt  
# Info 0 [plugin/confinement/src/model_hard_shape.cpp:21] args file_name slab.txt 
ThermoParams beta 1.5 chemical_potential0 1  
Metropolis  
TrialTranslate tunable_param 2 weight 1  
TrialAdd particle_type 0  
Run until_num_particles 50  
# initializing random number generator with seed: 1653487009
RemoveTrial name TrialAdd  
Log file_name slab.log trials_per 1e4  
Movie file_name slab.xyz trials_per 1e4  
CheckEnergy trials_per 1e4  
Tune  
Run num_trials 1e5  
 
 exit: 0
# FEASST version: v0.19.0-95-g62bcb048ca-dirty-hwh/newtutorials
Configuration cubic_box_length 8 particle_type0 /home/hwh/feasst/forcefield/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
Potential Model

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!